In [18]:
import os 
import pandas as pd
from collections import defaultdict
import numpy as np 
import glob

Step 1. Get the all sample experiement longlist: 
Use cut and unique in bash - (could be done in the notebook) 



Step 2. Get the metadata and peptides: 
Done in bash with the command below 

Step 3. Get the formating of the fasta file done within this notebook 



Step 4. Get the peptide/ experiment map in this notebook

# Preprocess the annotation 

In [15]:
ann_path = '/cluster/work/grlab/projects/projects2020_OHSU/annotation/gencode.v32.annotation.gtf'

In [16]:
# From Immunopepper preprocess.py 
def attribute_item_to_dict(a_item, file_type, feature_type):
    """  From attribute item in annotation file to get corresponding dictionary

    Parameters
    ----------
    a_item: str. attribute item
    file_type: str. Choose from {'gtf', 'gff', 'gff3'}
    feature_type: str. Extract other fields. We only
        consider 'CDS', 'mRNA' and 'transcript'

    Returns
    -------
    gtf_dict: dict. store all the necessary data

    """    
    
    gtf_dict = {}
    if file_type.lower() == 'gtf':
        attribute_list = a_item.split('; ')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split(' ')
            gtf_dict[pair[0]] = pair[1][1:-1]
    elif file_type.lower() == 'gff3':
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gtf_dict[pair[0]] = pair[1]
    elif file_type.lower() == 'gff':
        gff_dict = {}
        attribute_list = a_item.split(';')
        for attribute_pair in attribute_list:
            pair = attribute_pair.split('=')
            gff_dict[pair[0]] = pair[1]  # delete "", currently now work on level 2
        if feature_type == 'CDS':
            gtf_dict['transcript_id'] = gff_dict['Parent']
        elif feature_type in {'mRNA', 'transcript'}:  # mRNA or transcript
            gtf_dict['gene_id'] = gff_dict['geneID']
            gtf_dict['transcript_id'] = gff_dict['ID']
            gtf_dict['gene_type'] = gff_dict['gene_type']
            gtf_dict['transcript_type'] = gff_dict['transcript_type']

    return gtf_dict

In [17]:
#  From Immunopepper preprocess.py 
def leq_strand(coord1, coord2, strand):
    if strand == "+":
        return coord1 <= coord2
    else:
        return coord1 >= coord2


In [18]:
# From Immunopepper preprocess.py 
transcript_to_gene_dict = {}    # transcript -> gene id


gene_to_transcript_dict = {}    # gene_id -> list of transcripts
gene_cds_begin_dict = {}        # gene -> list of first CDS exons

transcript_to_cds_dict = {}     # transcript -> list of CDS exons
transcript_cds_begin_dict = {}  # transcript -> first exon of the CDS
transcript_to_strand = {}

file_type = ann_path.split('.')[-1]
chromesome_set = set()
# collect information from annotation file
for line in open(ann_path, 'r'):
    if line[0] == '#':
        continue
    item = line.strip().split('\t')
    chromesome_set.add(item[0])
    feature_type = item[2]
    attribute_item = item[-1]
    attribute_dict = attribute_item_to_dict(attribute_item, file_type, feature_type)
    # store relationship between gene ID and its transcript IDs
    if feature_type in ['transcript', 'mRNA']:
        gene_id = attribute_dict['gene_id']
        transcript_id = attribute_dict['transcript_id']
        if attribute_dict['gene_type'] != 'protein_coding' or attribute_dict['transcript_type']  != 'protein_coding':
            continue
        assert (transcript_id not in transcript_to_gene_dict)
        transcript_to_gene_dict[transcript_id] = gene_id
        if gene_id in gene_to_transcript_dict and transcript_id not in gene_to_transcript_dict[gene_id]:
            gene_to_transcript_dict[gene_id].append(transcript_id)
        else:
            gene_to_transcript_dict[gene_id] = [transcript_id]
        # Todo python is 0-based while gene annotation file(.gtf, .vcf, .maf) is one based
    elif feature_type == "CDS":
        parent_ts = attribute_dict['transcript_id']
        strand_mode = item[6]
        cds_left = int(item[3])-1
        cds_right = int(item[4])
        frameshift = int(item[7])
        transcript_to_strand[parent_ts] = strand_mode
        if parent_ts in transcript_to_cds_dict:
            transcript_to_cds_dict[parent_ts].append((cds_left, cds_right, frameshift))
        else:
            transcript_to_cds_dict[parent_ts] = [(cds_left, cds_right, frameshift)]
        if strand_mode == "+" :
            cds_start, cds_stop = cds_left, cds_right
        else:
            cds_start, cds_stop = cds_right, cds_left

        # we only consider the start of the whole CoDing Segment
        if parent_ts not in transcript_cds_begin_dict or \
           leq_strand(cds_start, transcript_cds_begin_dict[parent_ts][0], strand_mode):
            transcript_cds_begin_dict[parent_ts] = (cds_start, cds_stop, item)

# collect first CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    if target_gene not in gene_cds_begin_dict:
        gene_cds_begin_dict[target_gene] = []
    if ts_key in transcript_cds_begin_dict:
        gene_cds_begin_dict[target_gene].append(transcript_cds_begin_dict[ts_key])


In [19]:
# NEED TO SORT THE TRANSCRIPT TO CDS 

In [20]:
# Custom collection of CDS 
transcript_cds_begin_dict_bis = {}
transcript_cds_end_dict_bis = {}

gene_cds_begin_dict_bis = defaultdict(list)
gene_cds_end_dict_bis = defaultdict(list)

# will be in reading order 
for ts_key in transcript_to_cds_dict:
    if transcript_to_strand[ts_key] == '+': # '+'
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][0],
                                                 transcript_to_cds_dict[ts_key][0][1], '+')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][0],
                                                 transcript_to_cds_dict[ts_key][-1][1], '+')
        

    else: 
        transcript_cds_begin_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][0][1],
                                                 transcript_to_cds_dict[ts_key][0][0], '-')
        transcript_cds_end_dict_bis[ts_key] = (transcript_to_cds_dict[ts_key][-1][1],
                                                 transcript_to_cds_dict[ts_key][-1][0], '-')
    
    assert(transcript_cds_begin_dict_bis[ts_key][0] == transcript_cds_begin_dict[ts_key][0])
    assert(transcript_cds_begin_dict_bis[ts_key][1] == transcript_cds_begin_dict[ts_key][1])

# collect first, last CDS exons for all transcripts of a gene
for ts_key in transcript_to_gene_dict:
    target_gene = transcript_to_gene_dict[ts_key]
    gene_cds_begin_dict_bis[target_gene].append(transcript_cds_begin_dict_bis[ts_key])
    gene_cds_end_dict_bis[target_gene].append(transcript_cds_end_dict_bis[ts_key])

In [21]:
#gene_cds_begin_dict_bis

# Step 3. Format the peptide raw file (from step 2)

In [22]:
# fields_meta_peptide_dict = ['peptide', 'id', 'readFrame', 'geneName', 'geneChr', 'geneStrand',
#                                 'mutationMode',
#                                 'junctionAnnotated', 'hasStopCodon', 'isInJunctionList',
#                                 'isIsolated', 'variantComb', 'variantSegExpr', 'modifiedExonsCoord',
#                                 'originalExonsCoord', 'vertexIdx',
#                                 'kmerType']

In [23]:
file_meta = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/TCGA-A2-A0D2-01A-21R-A034-07.all_d4aee54_GTEXcore_kmer_peptides_raw.tsv'

In [240]:
file_save = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/G_TCGA-A2-A0D2-01A-21R-A034-07_query_peptides.fa'


In [38]:
## Format and create flags 
pep_idx = 0 


with open(file_save, 'a') as sp:
    with open(file_meta, 'r') as fp:
        print("open {}".format(file_meta))
        lines = fp.readlines()
        ### Iterate over kmers 
        for line in lines:
            if 'modifiedExonsCoord' in line: #header
                continue
            line = line.replace('3-exons_9-mer ', '3-exons_9-mer@').replace('2-exons ', '2-exons@')
            kmer = line.split(',')[0]
            peptides = ','.join(line.split(',')[1:])
            peptides = peptides.split('@')
            ### Iterate over kmer-associated peptides
            for pep in peptides:
                pep = pep.split(',')
                pep_seq = pep[0]
                pep_gene = pep[3]
                pep_orig_coordS = pep[13].split('/')
                pep_modif_coordS = pep[14]
                strand = pep[5]
                has_stop_codon = pep[8]
                if has_stop_codon == '0':
                    break 
                ### Iterate over original genomic coordinates for include flag
                for pep_orig_coord in pep_orig_coordS:
                    pep_orig_coord = pep_orig_coord.split(';')
                    pep_orig_coord = [coord for coord in pep_orig_coord if coord != 'None']

                    start_cds = [ first_exon[0] for first_exon in 
                                 gene_cds_begin_dict_bis[pep_gene] ] 
                    end_cds = [ last_exon[1] for last_exon in 
                               gene_cds_end_dict_bis[pep_gene] ] 
                    ## Get peptide end and start coordinates
                    if strand == '+': # Do - strand 
                        pep_start = np.int(pep_orig_coord[0])
                        pep_end = np.int(pep_orig_coord[-1])
                    else: 
                        pep_start = np.int(pep_orig_coord[1])
                        pep_end = np.int(pep_orig_coord[-2])

                    ## Use end and start coordinates for 3' 5' include flag
                    if pep_start in start_cds: # We will always miss things that are new in the graph 
                        pep_5include = 1
                    else: 
                        pep_5include = 0 
                    if (pep_end in end_cds) or (has_stop_codon) == '1':
                        pep_3include = 1
                    else: 
                        pep_3include = 0 

                ### Iterate over original genomic coordinates for the junction position flag and between codon flag
                for pep_modi_coord in pep_modif_coordS.split('/'):
                    pep_idx +=1
                    pep_modi_coord = pep_modi_coord.split(';')
                    pep_modi_coord = [coord for coord in pep_modi_coord if coord != 'None']
                    tot_len = 0 
                    jx_list = []
                    shift = 0 
                    shift_list = []
                    ## Get nt length of each exon involved -> jx_list, shift_list
                    for pair in np.arange(0, len(pep_modi_coord), 2):
                        cds = int(pep_modi_coord[pair + 1]) - int(pep_modi_coord[pair])  # 0 based, open right 
                        cds += shift 
                        shift = cds % 3
                        jx_list.append(cds - shift)
                        shift_list.append(shift)
                    kmer_len = 9 

    #                 for start_pos in np.arange(0, len(pep_seq) - (kmer_len - 1) ):
    #                     if pep_seq[start_pos : start_pos + kmer_len ] == kmer:
    #                         pos_kmer = start_pos # 0 based, real position = pos_kmer +1
    #                 assert(pos_kmer == jx_list[0]/ 3)

                    ## Get aa position of the junction
                    if shift_list[0]: # junction is inside an amino acid
                        aa_junction_pos = int((jx_list[0] / 3)) # because 0 based
                        between_codons = 0 
                    else: # junction is between amino acids 
                        aa_junction_pos = int((jx_list[0] / 3) - 1)  # because 0 based
                        between_codons = 1
                    if strand == '+':
                        genome_junction_pos = '{}_{}'.format(pep_modi_coord[1], pep_modi_coord[2])
                    else:
                        genome_junction_pos = '{}_{}'.format(pep_modi_coord[0], pep_modi_coord[3])
                    
                    
                    pep_handle = '>pepID-{};jx_pos-{};between_codons-{};includes_5\'-{};includes_3\'-{};gene-{};jx_coord-{}'.format(
                    pep_idx, aa_junction_pos, between_codons, pep_5include, pep_3include, pep_gene, genome_junction_pos)
                    sp.write(pep_handle + '\n')
                    sp.write(pep_seq + '\n')
               


open /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/TCGA-A2-A0D2-01A-21R-A034-07.all_d4aee54_GTEXcore_kmer_peptides_raw.tsv


# Step 4. Get the experiement/ peptide map 

## Step a. Get the experiment IDS

In [219]:
sample = 'TCGA-A2-A0D2-01A-21R-A034-07.all'
background = 'commit_d4aee54_GTEXcore'

In [220]:
experiment_folder = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_{}/{}'.format(sample, background)

In [221]:
experiement_files = glob.glob(experiment_folder + '/*Uniprot*tsv')

In [222]:
folder_OHSU = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/Aug21_graph_data_updatedfilters'

In [223]:
exp_map = pd.read_csv(os.path.join(folder_OHSU, 'J_experiment_map.tsv'), sep = '\t')

In [224]:
exp_map.head()

,id,motif_filter,min_sample_reads,#_of_cohort_samples,reads_per_cohort_sample,normal_cohort_id,#_normal_samples_allowed,reads_per_normal_sample
0,1,1,0,0,0,paired,0,0
1,2,1,0,0,0,paired,2,3
2,3,1,0,0,0,paired,2,10
3,4,1,0,0,0,paired,2,-1
4,5,1,0,0,0,paired,10,3


In [225]:
def path_to_condition(all_paths):
    '''Converts ETH Names into experimental conditions '''
    # None handling added 
    
    #all_paths = [path.replace('None', '-1') for path in all_paths]
    sample_expr = [os.path.basename(path).split('ref_SampleLim')[1].split('Cohort')[0]
                   for path in all_paths ]

    foreground_cohort_expr = [os.path.basename(path).split('CohortLim')[1].split('Across')[0]
                                       for path in all_paths ]

    foreground_cohort_samples = [os.path.basename(path).split('Across')[1].split('_Filt')[0] 
                                 for path in all_paths ]

    background_cohort_expr = [os.path.basename(path).split('Cohortlim')[1].split('Across')[0]
                              for path in all_paths ]

    background_cohort_samples = [os.path.basename(path).split('Across')[2].split('_FiltUn')[0]
                                 for path in all_paths ]
    background_cohort_id = [os.path.basename(path).split('Normals')[1].split('lim')[0] for path in all_paths ]

    legend_quant = pd.DataFrame({'min_sample_reads':sample_expr, 
                  'reads_per_cohort_sample': foreground_cohort_expr, 
                  '#_of_cohort_samples': foreground_cohort_samples, 
                  'reads_per_normal_sample': background_cohort_expr,
                  '#_normal_samples_allowed' : background_cohort_samples, 
                    'normal_cohort_id':background_cohort_id, 
                    'original_name':all_paths})

    return legend_quant

In [226]:
experiement_files

['/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/G_TCGA-A2-A0D2-01A-21R-A034-07.all_ref_SampleLim2.0CohortLim5.0Across10_FiltNormalsGtexcoreCohortlim0.0Across0_FiltUniprot.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/G_TCGA-A2-A0D2-01A-21R-A034-07.all_ref_SampleLim0.0CohortLim0.0Across2_FiltNormalsGtexcoreCohortlimNoneAcross2_FiltUniprot.tsv',
 '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102/filter_TCGA-A2-A0D2-01A-21R-A034-07.all/commit_d4aee54_GTEXcore/G_TCGA-A2-A0D2-01A-21R-A034-07.all_ref_SampleLim0.0CohortLim5.0Across1_FiltNormalsGtexcoreCohortlim3.0Across2_FiltUniprot.tsv',
 '/cluster/

In [227]:
path_to_table = path_to_condition(experiement_files)


In [228]:
path_to_table.loc[path_to_table['min_sample_reads'] == 'None', 
                  'min_sample_reads'] = 0
path_to_table.loc[path_to_table['reads_per_cohort_sample'] == 'None', 
                  'reads_per_cohort_sample'] = 0
path_to_table.loc[path_to_table['#_of_cohort_samples'] == 'None', 
                  '#_of_cohort_samples'] = 0
path_to_table.loc[path_to_table['reads_per_normal_sample'] == 'None', 
                  'reads_per_normal_sample'] = -1
path_to_table.loc[path_to_table['#_normal_samples_allowed'] == 'None', 
                  '#_normal_samples_allowed'] = -1

In [229]:
exp_map.loc[exp_map['normal_cohort_id'] == 'core_GTEx', 'normal_cohort_id'] = 'GtexcoreCohort'

In [230]:
path_to_table['motif_filter'] = 0 

In [231]:
for col in path_to_table.columns: 
    if col != 'original_name':
        print(col)
        print(exp_map[col].dtypes)
        print(path_to_table[col].dtypes)
        if (exp_map[col].dtypes != path_to_table[col].dtypes) :
            print('change type')
            if exp_map[col].dtypes == 'int64':
                path_to_table[col] = path_to_table[col].astype( 'float64' )
            path_to_table[col] = path_to_table[col].astype( exp_map[col].dtypes ) 
        print('\n')

min_sample_reads
int64
object
change type


reads_per_cohort_sample
int64
object
change type


#_of_cohort_samples
int64
object
change type


reads_per_normal_sample
int64
object
change type


#_normal_samples_allowed
int64
object
change type


normal_cohort_id
object
object


motif_filter
int64
int64




In [232]:
exp_map.columns

Index(['id', 'motif_filter', 'min_sample_reads', '#_of_cohort_samples',
       'reads_per_cohort_sample', 'normal_cohort_id',
       '#_normal_samples_allowed', 'reads_per_normal_sample'],
      dtype='object')

In [233]:
path_to_table.columns

Index(['min_sample_reads', 'reads_per_cohort_sample', '#_of_cohort_samples',
       'reads_per_normal_sample', '#_normal_samples_allowed',
       'normal_cohort_id', 'original_name', 'motif_filter'],
      dtype='object')

In [234]:
id_mapped = path_to_table.merge(exp_map, on = ['min_sample_reads', 'reads_per_cohort_sample', \
                                   '#_of_cohort_samples', 'reads_per_normal_sample', \
                                   '#_normal_samples_allowed',
                                   'normal_cohort_id', 'motif_filter'], how = 'inner')

In [235]:
id_mapped.shape

(120, 9)

In [236]:
id_mapped.head()

,min_sample_reads,reads_per_cohort_sample,#_of_cohort_samples,reads_per_normal_sample,#_normal_samples_allowed,normal_cohort_id,original_name,motif_filter,id
0,2,5,10,0,0,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,475
1,0,0,2,-1,2,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,388
2,0,5,1,3,2,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,380
3,0,1,10,-1,2,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,412
4,0,1,10,0,0,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0,409


In [237]:
# path_to_table.loc[(path_to_table['min_sample_reads'] == 0) & \
#             (path_to_table['reads_per_cohort_sample'] == 0) & \
#            (path_to_table['#_of_cohort_samples'] == 2) & \
#            (path_to_table['reads_per_normal_sample'] == -1) & \
#            (path_to_table['#_normal_samples_allowed'] == 2) & \
#            (path_to_table['normal_cohort_id'] == 'GtexcoreCohort') ]

,min_sample_reads,reads_per_cohort_sample,#_of_cohort_samples,reads_per_normal_sample,#_normal_samples_allowed,normal_cohort_id,original_name,motif_filter
1,0,0,2,-1,2,GtexcoreCohort,/cluster/work/grlab/projects/projects2020_OHSU...,0


## Step b.Map the longlist of peptides to their experiements

In [ ]:
# Fasta Read
fasta = {}
peps = set()
with open(file_save, 'r') as fp:
    for line in fp.readlines():
        if '>' in line:
            handle = line
        else: 
            fasta[handle] = line
            peps.add(line)
            
        

In [ ]:
kmers_dict = [[],[], []]
with tarfile.open(tar_file_OHSU, "r:*") as tar:
    for io, _ in enumerate(file_pair['eth']):
        #print(io)
        name_ohsu = file_pair['ohsu'][io]
        eth_path = file_pair['eth'][io]
        df_ohsu = pd.read_csv(tar.extractfile(name_ohsu), header=0, sep="\t")
        df_eth = pd.read_csv(eth_path, sep="\t", usecols = ['kmer'])
        df_eth = set(df_eth['kmer'])
        df_ohsu = set(df_ohsu['kmer'])
        for kmer in df_eth:
            kmers_dict[0].append(kmer)
            kmers_dict[1].append(eth_path.split('/')[-2])
            kmers_dict[2].append(io)
        for kmer in df_ohsu:
            kmers_dict[0].append(kmer)
            kmers_dict[1].append(name_ohsu)
            kmers_dict[2].append(io)


In [248]:
len(fasta)

41877

In [ ]:
fasta_ 

In [244]:
fasta.head()

,0
0,>pepID-1;jx_pos-12;between_codons-1;includes_5...
1,WLRGELDRAAADGWTMGTGARLSPSCVPGTWTGETCVPSSRRPWWR...
2,>pepID-2;jx_pos-27;between_codons-1;includes_5...
3,ATVGSSILALLCREVAPSRWGVAAAITGLLAVM
4,>pepID-3;jx_pos-6;between_codons-1;includes_5'...
